# Run streamlit app from a Google Colab Notebook


In [1]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.5 MB/s eta 0:00:00


download preptrained model and dataset
### also overrides when if downloaded again

In [2]:
import os
if not os.path.exists("oxford-iiit-pet.zip"):
  os.system('curl -O https://dlcv2023.s3.eu-north-1.amazonaws.com/oxford-iiit-pet.zip')
if not os.path.exists("resnet18_finetuned.pkl"):
  os.system('curl -O https://dlcv2023.s3.eu-north-1.amazonaws.com/resnet18_finetuned.pkl')

In [3]:
# install needed dependancies
%%capture
%%bash
#pip install fastai2
# install captum from repo
git clone https://github.com/tendermonster/captum.git
pip install captum/ fastai2 psutil

In [6]:
from matplotlib.colors import LinearSegmentedColormap
import numpy as np
from fastai.vision.all import PILImage, load_learner, untar_data, get_image_files, ImageDataLoaders, ClassificationInterpretation, Resize, cnn_learner, URLs, resnet18, error_rate
from captum.attr import IntegratedGradients
from captum.attr import GradientShap
from captum.attr import Occlusion
from captum.attr import Saliency
from captum.attr import NoiseTunnel
from captum.attr import visualization as viz

In [ ]:
import os
if not os.path.exists("./resnet18_finetuned.pkl"):
  # if file does not exist finetune resnet18 on the dataset
  # finetune the model on the PETS dataset
  learn = cnn_learner(dls, resnet18, metrics=error_rate)
  learn.fine_tune(epochs=1)
  # Basic interpertation of the model prediected classes vs. actual ones.
  interp = ClassificationInterpretation.from_learner(learn)
  # Visualize top losses, e.g. to check if the images themselves are correctly annotated.
  interp.plot_confusion_matrix()
  interp.plot_top_losses(5, nrows=1)
  # save the model
  learn.export('./resnet18_finetuned.pkl')
else:
  # load the model
  #learn_inf = load_learner(cpu=False, fname='./resnet18_finetuned.pkl')

IndentationError: ignored

## Create a streamlit app example


In [26]:
%%writefile app.py
import streamlit as st
import random
from fastai.vision.all import PILImage, load_learner, untar_data, get_image_files, ImageDataLoaders, ClassificationInterpretation, Resize, cnn_learner, URLs, resnet18, error_rate

path = './oxford-iiit-pet/images'
imgs = get_image_files(path)

idx = random.randint(0, len(imgs))
images = PILImage.create(imgs[idx])
image = st.image(images)


Overwriting app.py


## Install localtunnel

In [27]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.641s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



## Run streamlit in background

In [28]:
!streamlit run /content/app.py &>/content/logs.txt &

## Expose the port 8501
Then just click in the `url` showed.

A `log.txt`file will be created.

In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 1.544s
your url is: https://large-houses-call-34-75-24-52.loca.lt


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
